Using this to write out my resume, then copying it. 

Would be a fun project to do this automatically, now that I have all the styling etc. for it :) 

# Daniel McCann-Sayles' Portfolio

### Summary

<p style="display: flex; justify-content: space-between; right-margin: 50px ">
  <span>Cisco</span>
  <span ">thing2</span>
</p>

Summary :p

### Work Experience

Cisco - Intern

### Projects

#### Kid Fight Game

My friend Devin is interested in game design. I was talking with him about how having ‘real AI’ in games is going to allow awesome interactive stuff. I thought it would be fun to see what could be done creatively with AI

As a kid I used to have arguments with friends where we would one up the other. I would have a laser, he would have a laser deflecting mirror, I would have gone back in time and weakened his mirror so it cracked, etc. 

I thought it would be fun to approximate some of this back and forth creativity - in a table top manner, like Pokemon.
There were two distinct phases to this project.

I'll talk a bit about each, but only show code for the second. You can check out the repository here *TODO*


##### First pass / Pygame era:

We first started by finding a pygame game implementing chess. It made sense to just start with that chessboard. Unforunately this introduced a pattern of coupled game display and logic that lasted for a while. 

This was pretty fun. Lots of learning how to use pygame. Learned about pygame events to allow certain things to happen in time w/ clock. For calling chatgpt we did this asynchronously. To do this I made a new thread, which is debatably the right answer, but it works fine. We started implementing some path finding and logic for the little game pieces. 

I learned a fair amount about OOP as well, something I'm not super familiar with. Pygame is cool, but it also took a fair amount of boilerplate to make something simple. Though maybe this is just all game development -> not my field :).


##### Second pass / Simulation era:


I ended up being busy for a bit, but when I came back I had a fresh perspective. I realized that this would never be fun/useful unless two people could type, which necessitates multiplayer. So I set up a simple network protocol using the socket library. 

This also led me to decouple the display logic and the game logic, and cleared up a lot. Instead of having all these pygame classes that handled their state and display, I have all that logic in python classes on the server. I communicate the board state through a json object, and have pygame classes on the frontend/client. You can see below the seperation of logic - the client cares about looks, and the server cares about functionality.

In [ ]:
### Client implementation of Board
class Board:
    def __init__(self, size: int, x_offset, y_offset, rows: int):
        self.width = size
        self.height = size
        self.x_offset = x_offset
        self.y_offset = y_offset
        self.square_size = size // rows
        self.rows = rows
        self.squares: list[Square] = self.generate_squares()

    def update_board(self, board_list: list[str | PieceData]):
        """Update board from json data. Clear everything on squares, then re-add"""
        if len(self.squares) != len(board_list):
            print(
                "This should not happen.. mismatch btwn server and client ",
                len(self.squares),
                len(board_list),
            )

        for data, square in zip(board_list, self.squares):
            square.occupying_piece = None
            square.highlight = False

            if is_piece_data(data):
                piece = Piece(self.square_size, data)
                square.occupying_piece = piece

            elif data == "H":
                square.highlight = True

    def draw(self, display):
        for square in self.squares:
            square.draw(display)
            
    ...

In [ ]:
### Server implementation of Board
class Board:
    def __init__(self, rows: int):
        self.rows = rows
        self.squares: list[list[Square]] = self.generate_squares()
        self.bases = self.make_bases()
        self.characters: dict[str, list[Character]] = {"black": [], "white": []}

    def to_json(self):
        """Got through the squares and serialize each of them and return it. Returns a flattened list"""
        flat_squares = [square for columns in self.squares for square in columns]
        json_list = [square.to_json() for square in flat_squares]

        return json_list

    def generate_squares(self):
        output: list[list[Square]] = []
        for row in range(self.rows):
            inner_list = []
            for column in range(self.rows):
                inner_list.append(Square(row, column))

            output.append(inner_list)
        return output

    def add_piece_safe(self, pos: tuple[int], piece: DefaultPiece):
        """Add the given piece to the given position on the board. Fails quietly w/ console log. Sets the pieces position to square position"""
        square = self.get_pos(pos)
        if not square:
            print("invalid square", pos)
            return
        if self.check_if_occupied(square):
            print("already occupied")
            return

        square.occupying_piece = piece
        piece.set_pos((square.row, square.column))

    ...


They communicate through a simple network protocol, using sockets.

`mserver.py` hosts a server which starts threaded processes as clients join. `mclient.py` initiates a new pygame (`SrverGame.py`) process that uses the Network (`network.py`) class to communicate with the server. 

Below are snippets from each to hopefully convey their main functions.

In [ ]:
# mserver
import socket
from _thread import start_new_thread
from client.helpers import get_mock_board
import json
from server.ServerGame import ServerGame
from address import SERVER_ADDRESS

# Class to handle game logic. 
server_game = ServerGame()

# Global to communicate whether players are here or not
players = {"black": False, "white": False}

def threaded_client(conn: socket.socket, color: str):
    # Start connection by sending initial obj
    initial = json.dumps(
        {"loading": False, "board": server_game.board.to_json()}
    ).encode()
    conn.send(initial)

    while True:
        try:
           # Recieve Data ..

            # If we get data saying we want a new character
            if "description" in json_data:
                server_game.create_character(color, json_data["description"])
                print("recieved description ", json_data["description"])

            # Call this every iteration. handles game logic & and other stuff
            board_json, loading = server_game.gameloop(color)

            # Return 
            conn.sendall(json.dumps({"loading": loading, "board": board_json}).encode())
        except Exception as e:
           break

    print("Lost connection to ", color)
    players[color] = False
    conn.close()


""" Handle new connections"""
while True:
    p1 = players["black"]
    p2 = players["white"]

    conn, addr = s.accept()

    # Add players ..

In [ ]:
#mclient => ServerGame.py
import time
import threading
from llm.call_gpt import generate_character_stats_multiplayer
from server.classes.Board import Board
from server.classes.Character import Character


class Clock:
    def __init__(self, interval, stopped=False):
        """Interval in seconds"""
        self.interval = interval
        self.last_time = time.time()
        self.stopped = stopped

    def check(self):
        """Returns true if interval has passed"""

    ...


class ServerGame:
    def __init__(self):
        self.main_clock = Clock(1)
        self.sub_clock = Clock(1, stopped=True)

        # Board object for representation.
        self.board = Board(10)

        # Lock prevents main check from running twice (called in two threads)
        self.lock = threading.Lock()

    def callback(self, response, color):
        """Callback is a method to have access to self - called in thread to get async GPT"""
        self.character_to_add[color] = Character(
            (0, 0),
            color=color,
            board=self.board,
            attack_dmg=response["AD"],
            hp=response["HP"],
            move_distance=response["MD"],
        )

    def create_character(self, color, description: str):
        "Call gpt to create a character. Currently making a new thread to do so. daemon means it will be terminated"
        if self.loading_character[color]:
            return

        self.loading_character[color] = True
        description = description or "bland default character"

        threading.Thread(
            target=generate_character_stats_multiplayer,
            args=(description, color, self.callback),
            daemon=True,
        ).start()

   

    def gameloop(self, color):
        """Runs constantly in while loop"""
        if self.main_clock.check():
            # Do stuff..

        if self.sub_clock.check():
            self.move_character()

        # Check if we need to add a character
        character = self.character_to_add[color]
        if character:
            # Add character..

        # Return current board
        return self.board.to_json(), self.loading_character[color]

The original idea of this game was to explore the added creativity an LLM could bring. In this case, the idea was to have an impartial judge add or subtract to bonuses to characters depending on the existing characters on the board.

A next step was to add behavior, and then to add special moves, but we did not get around to this.
Instead we were pretty sidetracked by building out the main game engine functionality.
Anyways heres the general prompt/schema we hoped to follow.

Of some note is the Strength section. We wanted to allow the LLM judge to give bonuses to characters against other characters specifically. 

In [ ]:
CHARACTER_PROMPT = """
You are a character creator. Your job is to determine the stats of a character based on these two things:
1. A list of existing characters on the board (a list of json objects, of the format {id: <int>, description: <str>}) 
2. A description of the character passed in (a string)

These stats are as follows:
1. HP - Health points. This is not to exceed 10, or go below 1. Some examples for reference:
    A knight in armor might have a high HP, like 6. An archer might have a low HP, like 3. A mouse might have a tiny HP, like 1.

    ...
"""

API_SCHEMA = {
    "name": "character_stats",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {
            "HP": {"type": "integer"},
            "AD": {"type": "integer"},
            "MD": {"type": "integer"},
            "Behavior": {"type": "integer"},
            "Strength": {
                "type": "object",
                "properties": {
                    "against_id": {"type": ["integer", "null"]},
                    "modifier": {"type": ["integer", "null"]},
                },
                "required": ["against_id", "modifier"],
                "additionalProperties": False,
            },
        ...
}}}

Reflection:

1. Learned more about async and anyio, network protocol, and OOP w/ Python
2. It's important to always decouple logic & display - it would've been much easier to refactor for the simulation if I had followed this practice from the beginning

Next steps:

In hindsight, we tried to tackle two things at the same time, and it would have been better to only tackle one. Either a =>
1. creative response game using AI to evaluate that creativity
2. simulator/board game, with physical pieces and a map

I’d actually like to make the first one right now - instead of a physical board, using some sort of card game, Magic-like system would be much better for a prototype of on the fly, AI-judged creativity. I have some other things to do first though ..

This leads to one final lesson learned, which is common in many of my projects..
> simplify prototypes & identify measurable objectives! BEFORE starting

#### MACM exploration

During some downtime at work, I explored a Cisco research project focused on agentic AI workflows to improve performance on math problem-solving benchmarks. 

I optimized the code base, explored asynchronous alternatives, and reduced costs while increasing speed. I  also explored alternative methods for connecting LLM calls and evaluated their effectiveness in problem-solving.

This experience sharpened my skills in prompt engineering, OpenAIs API structure, and agentic workflows (specifically debugging & dealing with overhead).

I wanted to explore chaining multiple calls to LLMs. I started with this [paper](https://arxiv.org/abs/2404.04735) as an existing benchmark. In the paper they essentially just chain a few LLM calls together and get some performance improvement on a benchmark of math problems. Their methodology essentially involves: 

1. Calling the LLM to examine the initial problem
2. Repetitively asking the LLM for `conditions` that could be helpful to solve it. 
3. Doing step 2 5 times to make a pool.
4. Asking the LLM to generate solution steps using the condition pool.

They also try and give the LLMs roles, with a judge, executor, and ideator, but I don't think this matters.

My goal was this project was just to learn, and mess around, using what they have as a baseline. Here is a rough timeline of what I attempted:

1. First, I cloned and looked at the existing code base
    1. I re-wrote a lot of their stuff to be modifiable, and added helper functions.
    2. I converted most of the GPT calls to async (I didn't finish this until later). For reference, the first time I ran it on one problem it took about 7 minutes (19 in-order requests). 
    3. I cleaned up prompts, and I switched the whole thing to a cheaper form of GPT so I could run multiple times. 
    4. I also moved away from using the code editor, and I switched to pure LLM calls instead of using the OpenAI assistants. 
        > I actually spent a while messing around with this. They were using OpenAI's Assistants. OpenAI charges per code session, but once you start a code session it lasts for an hour. So if I could re-use this code session it would be cheaper. This required me to use the same conversation, so I set up some helper code to manage clearing and re-populating the message history. I finally got it working, and it only saved me ~3% of the total cost : /
    4. I added logging so that I could start investigating what was going wrong.
    5. By switching to just calling chat complete instead of using assistants I was able to enable schema enforcement. This allowed me to get exact types from GPT responses. This was *way* nicer than using regex and hoping. Of course, in the background all OpenAI is doing is some form of regex/heuristics + some sort of self-correction - BUT they have much more error handling than me. Why do it myself?

In [ ]:
#Should I put some code here??
# * some async stuff
# * assistant code session stuff
# * loggin


*At this point it would have been a good idea to re-run benchmarks. I did not do this. This led to having a hard time measuring improvement, since my only comparison was their original code, which performs better since but uses expensive GPT and drew many more samples.* 

2. Then I started tweaking things.  
    1. I didn't like how they were doing the same thing over and over and averaging it. It's possible this is actually the [best way](https://redwoodresearch.substack.com/p/getting-50-sota-on-arc-agi-with-gpt) but I didn't find it particularly interesting. Instead I wanted to try doing more correction of wrong steps - finding errors and trying to fix them directly. 
        1. Multiple checks on conditions - check if we have generated enough, and then check that. 
        2. Re-assessmemnt of solution steps + regeneration

In [ ]:
# CODE 
# * Show small additions. Consider showing diff somehow?

3. I noticed that the conditions as they were were not really conditions. They were basically just tyring to solve the problem. Half the time they would be steps to solve the problem. So basically just *drawing more samples* again. To do something different I wanted to do something more in order. When solving a math problem, I'd be doing different steps each time. I completely switched up the idea =>
    1. Ask the LLM to generate a step by step solution to the problem in one pass.
    2. Go through step by step
    3. Verify that the step is correct. If not, regenerate it and the following steps.
    4. Once we reach the end, try having the LLM follow those steps.

*This is the graphic shown :)*

In [ ]:
# CODE - add one of the chains

This works okay. 

The code started getting a bit messy, and I got lost in terms of metrics. There were really two things that I had to manage - the overhead and building out helpers (logging, reusable functions) - and trying out different patterns of calling/prompts. It caused a fair amount of friction doing both. I see the value in tools like langchain - at least for when you're prototyping.

> If I was approaching this project now I would start by defining actual metrics, and ideas for success. Exploration is fun, but its hard to make progress when you don't know where you're going.

More detailed reflections: 

1. This endeavor was not entirely fruitful. One big challenge is benchmarking. Every call to GPT has some element of randomness in its return. When adding multiple together, it gets very complex very quickly. Even one call can be hard since there’s no function or obvious way to calculate randomness (that I know of). It’s also kind of a nebulous problem.

2. Another is how AI models are easily influenced. I imagine this is due to training that makes them agreeable to users, but when given a false idea (let’s say call #1 returned an incorrect assumption), it seems that call #2 often will not question that idea. 

Taking this idea a bit further, each call seems to perform worse and worse. I’m not sure why this is, though it would be interesting to read more about it. It seems impossible to do error correcting *completely* with LLM’s right now. 

In the end I did not achieve much noteworthy. But I understand a few things better now:
1. python codebases
2. python asynchronous libraries - I used anyio w/ the default configuration of asyncio backend. I like anyio.
3. The complexities that arise in multi-LLM interactions
4. The overhead that comes with trying to improve chained LLM performance - there are so many variables to adjust - models, prompts, chaining order, and much more. And it's not a consistent result. 

I also had some more abstract rambling:

I thought a lot about how we think, and about hiearchial forms of thought. For instance, starting at small provable concepts, using those to create higher level concepts, and so on. This is (very simplified) how we do things as humans. If I look at a complex belief I have, it’s predicated on lower and lower level ideas that have been lodged in my brain. It can be hard to change someones mind because even when a low level fact is changed, humans don't always update the higher order concepts that rest on it, or even think about that process. 

This leads me to something that I’ve been thinking about for a while, a way of creating a living essay. It would be cool to write a persuasive essay in a way that drew directly from the facts in it, so that when things were changed, everything downstream changed too. 

Of course, a lot of ideas are nebulous, but if you could lay out some amount of priors and assign some weight to each that showed how much it affected a higher level concept, then a reader could actually interact with your argument and see it change. 

#### AI Generation

** There are (at least) two obvious things wrong with this graph - one is that the dots overlap, meaning the last applied (gpt diff) is overrepresented. The other is that its not super helpful, except for a preliminary visualization. If I was still working on this, the next thing I would do would be sorting/clustering these data points based on how they do - e.g. these do well w/ the gpt diff, these do well with the semantic diff. 


This visualization would look cool, but would only be marginally more helpful, since it would end up with me diving back in to looking data point by data point. Ideally would be to find some way to extract similarities in the text itself, to avoid looking through each one myself. This idea of general pattern recognition reminds me of visual nn and makes me think I could train a simple model to predict how well a sample text would do on the different diff methods, and then extract the features it found. **

I had the opportunity to work on an internal Cisco project exploring how well an LLM email generation process was doing. A system would generate emails which are then looked over by users, optionally changed, and then sent. As a metric of how well the process was doing, the stakeholders were currently calculating a diff between the sent and generated email. 

I was tasked with exploring this diff, and seeing how it could be improved.

An important lesson I learned is that ‘data analysis’ is a surprisingly difficult thing to do. This motif appears later in my AI Math solving exploration as well. There are so many different ways of calculating metrics, and so many different metrics. 

Crucially, these metrics also depend on the audience and your intentions - the metrics you would show to an executive are completely different than those you might show to a technical stakeholder, and again different than those a customer might care about.

It's fascinating. You want to create some sort of visualization out of your data that your audience can easily read. This is something I find really interesting, especially when reading AI papers. 

To explore the diff, I created multiple Jupyter notebooks and spent a few weeks immersing myself in the data.

 1. I poured over hundreds of examples, explored different ways of both sorting & filtering, and visualizing the differences  to get impactful metrics. 

In [ ]:
# Display a toggleable output
from IPython.display import display, HTML

def toggleable_output(text, identifier):
    html = f"""
    <button onclick="var x = document.getElementById('toggleText_{identifier}'); x.style.display = x.style.display === 'none' ? 'block' : 'none';">
        Toggle {identifier}
    </button>
    <div id="toggleText_{identifier}" style="display:none;">
        {text}
    </div>
    """

    display(HTML(html))

In [ ]:
# Create html object to show diff visually
def show_visual_diff_difflib(
    old_string: str, new_string: str, wrap_column=60
):
    diff_html = difflib.HtmlDiff(wrapcolumn=wrap_column).make_file(old_string.split(), new_string.split(),)

    # Remove legend & links
    diff_html = re.sub(r'<table class="diff" summary="Legends".*?</table>', "", diff_html, flags=re.DOTALL)
    diff_html = re.sub(r'</td>\s*<td>\s*<table border="" summary="Links".*?</table>\s*</td>\s*</tr>', "", diff_html, flags=re.DOTALL)

    return '<style>.diff_add, .diff_chg, .diff_sub {color: black;}</style>' + diff_html

In [ ]:
# Narrow down to useful data to check out. When checking how a process affected a diff I couldn't just plot everything. It becomes a mess, as shown by the plot above
improved_diffs = []
for row in items: # Items is a df of calculated diff scores and text
    new_diff = row["New_diff"]
    diff = row["Diff"]
    if new_diff - diff > .2: # I played around with this number. 0.2 usually was a good heuristic of narrowing down the data but not nitpicking
        improved_diffs.append(row)

print(f"rows with improvements > .2: {len(improved_diffs)}, out of {len(items)}")

# For item in improved_diffs
# Using the difflib html object + toggling to make the visualization process easier
original_diff_text = f"Original diff:\n{item['Diff']}\n" + show_visual_diff_difflib(item["Generated"], item["Sent"])
toggleable_output(original_diff_text, "original_diff")

new_diff_text = f" diff:\n{item['new_diff']}\n" + show_visual_diff_difflib(item["new_diff_generated"], item["new_diff_sent"])
toggleable_output(new_diff_text, "new_diff")

 2. I tried multiple methods for cleaning the data before taking the diff, including using ChatGPT and semantic similarity matching, to get a score that reflected *more accurately* how good the generation was. 

In [ ]:
# Returns the original text split into main part and trailing addition
def check_if_trailing_addition(sent: str, generated: str):
    testing_diff = list(
        difflib.ndiff(
            generated.strip().splitlines(),
            sent.strip().splitlines(),
        )
    )
    last_section_length = 0

    # Find the last section of completely new lines
    for i, line in enumerate(reversed(testing_diff)):
        if line.startswith("+ "):
            last_section_length += 1
        elif line.startswith("? "):  # difflib metadata
            continue
        else:
            break

    if last_section_length > 0:
        # Split into main & trailing addition
        sent_lines = sent.strip().splitlines()
        main_part = "\n".join(sent_lines[:-last_section_length])
        trailing_addition = "\n".join(sent_lines[-last_section_length:])
        return trailing_addition, main_part

    return False, sent

In [ ]:
# Code to detect common signature chunks ----------
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')
signature_embeddings = model.encode(known_signature_chunks, convert_to_tensor=True)

# Pre-vectorize searchable text, broken down by sentence level
def vectorize_text_by_sentences(text: str, model: SentenceTransformer):
    sentences = text.splitlines()
    sentence_embeddings = model.encode(sentences, convert_to_tensor=True)
    return sentences, sentence_embeddings

def remove_signature_chunks(trailing_addition: str, signature_embeddings, model: SentenceTransformer, threshold=0.6):
    lines, sentence_embeddings = vectorize_text_by_sentences(trailing_addition, model)

     # Sliding window - to match and remove signatures
    for start in range(len(lines)):
        for window_size in range(1, len(lines) - start + 1):
            # Extract the embeddings for the current window of sentences
            window_embeddings = sentence_embeddings[start:start + window_size]
            combined_window_embedding = window_embeddings.mean(dim=0, keepdim=True)

            # Calculate cosine similarity between the current window and known signature embeddings
            cosine_scores = util.cos_sim(combined_window_embedding, signature_embeddings)
            max_score = max(cosine_scores[0])

            if max_score > threshold:
                print(f"Match: {max_score:.4f}")
               # Remove all lines after the match
                remaining_text = "\n".join(lines[:start])
                return remaining_text
    # No match found
    return False

Reflection: 

1. I made some progress on improving the diff, and exploring alternative methods. I found that the diff library they were using was comparing sentences, so by changing this to match every single character the similarity score increased - which was, *technically*, the objective. 

2. I learned a lot - becoming more familiar with python, the pitfalls of data analysis - it's easy to get lost in the minutia.  

3. Something that interested me, but I was not able to explore, was using the diff scores to improve the generation process. We have the data for each diff - it would be interesting to try collating these and sorting them by similar corrections to produce recommendations or common problems. You could imagine this being done automatically and generating a list of top 'x' priorities to make the generation more accurate, along with the % improvement each would bring. 

    > Whenever I can, I like to automate. This drive and desire to explore new ideas is something that I look forward to bringing to a smaller company, with more cross-functional opportunities



The lead of the project remarked:
"Your curiosity, ability to learn new concepts, and come up with out-of-the-box ideas are impressive.”

#### Speech to LLM App

I think well when walking, and I would like to be able to brainstorm ideas while in that flow state. Going back for my computer or pulling out my phone often interrupts what I'm thinking. It would be nice to have voice transcribed notes.

Additionally, I find that a conversational dialog is conducive to developing better ideas. I worked on this for a couple of weeks in July while I had some free time. 

The idea is simple ->
1. A basic iOS app that sends audio to a Node server on my computer. 

CODE

2. The Node server transcribes the audio and listens for a wake word. 

CODE


3. Upon this wake word a call to ChatGPT is triggered with the past `x` amount of context, and a response is generated.

CODE


4. The response is turned into audio, and sent to the app. The conversation transcription & response are stored as notes.

CODE


Some things I considered doing but didn’t get around to were:
1. Adding keywords that could create new notes, or give the context of the previous x lines of notes to ChatGPT, along with a command to format it in some way.
2. More generally, giving more options to ChatGPT - e.g. searching, or custom function calling. 

I stopped this project because I wasn't convinced I was actually getting more benefit out of it than going on a walk to clear my mind and then writing on my computer. 

I still think that a `perfect` AI assistant with transcription and note taking and function calling, *AND* a LLM re-writing the notes for clarity/ future reference, or some sort of mental model, would be *decently* useful.

Unforunately this would require:
1. Much more development & edge case debugging
2. A period of adaptation for me => I found that I mumble a lot, and directly turning my thoughts into words *coherently enough* is harder than I thought. 

And I was not ready to commit to both of these. 

Takeaways:
1. I do not enjoy Swift - However, I was able to hack together an app using a lot of ChatGPT and debugging. 
    
    > Tech stacks are just *tools* that allow you to do *things*. Doing *things* is what’s important. 
2. I didn’t have a measurable goal for this prototype stage - 
    1. As I mentioned above, the stage I reached was not super helpful to me, which meant I wasn't able to validate this project super well. 
    2. It’s important to build in measurable outcomes to prototype stages - how can I validate if this idea is *worth continuing*? What value can I derive from *each stage*

Overall, I learned a bit about calling LLMs, voice recording, audio processing, node Streams, and node servers.
got a good benchmark for how well I could do something technically unfamiliar with ChatGPT. 

Chips => LLMs, Node, Streams, Swift

In [ ]:

//helpers/rev-streaming.ts

// RevClient, revAiStream, and bufferStream are global variables passed in by reference, and modified in this function
export function startStreamingSession (revClient: RevAiStreamingClient | null, revAiStream: Duplex | null, bufferStream: PassThrough | null) {
    revClient = new RevAiStreamingClient(REV_AI_API_KEY, audioConfig)
    revAiStream = revClient.start()
  
    // Listen once for initialization
    let connectedPromise: Promise<{ connected: boolean; error?: string }> =
      new Promise((resolve, _) => {
        revClient!.once('connectFailed', error => {
          resolve({ connected: false, error })
        })
  
        revClient!.once('connect', connectionMessage => {
          console.log(
            `Connected with message: ${JSON.stringify(connectionMessage)}`
          )
          resolve({ connected: true })
        })
  
        setTimeout(() => {
          resolve({
            connected: false,
            error: 'Took longer than 5 seconds to initiate'
          })
        }, 5000)
      })
  
    //Add listeners to audio stream
    audioProcessing(revAiStream)
  
    // Buffer stream to handle incoming PCM data
    bufferStream = new PassThrough()
  
    // Pipe the duplex stream to our revAiStream so it can be accessed elsewhere
    bufferStream.pipe(revAiStream)
  
    return connectedPromise
  }

In [ ]:
//text_processing.ts

export function processTextListener (
  transcriptionDataObject: TranscriptionDataObject,
  revAiStream: Duplex,
  outputPaths: OutputPaths
) {
  // Process events received from Rev.ai transcript (response) stream. rev.ai returns 'partial' and 'final' chunks
  revAiStream.on('data', data => {
    console.log('*')
    if (data.type === 'partial') {
      const newWord: string = data.elements[data.elements.length - 1]
        .value as string
      transcriptionDataObject.partialData += newWord + ' '
    } else if (data.type === 'final') {
      let sentence = ''
      for (const element of data.elements) {
        sentence +=
          element.value +
          (element.type === 'punct' && element.value === '.' ? '\n' : ' ')
      }
      transcriptionDataObject.finalData += sentence
      transcriptionDataObject.partialData = '' 
    }

    // Call format - I was experimenting w/ having ChatGPT format the notes to fix transciption errors, of which there were many
    if (transcriptionDataObject.finalData.length >= 300) {
      formatDocument(transcriptionDataObject.finalData, outputPaths)
      transcriptionDataObject.finalData = '' 
    }
  })

  revAiStream.on('close', (err: any) => {
   
  })
}


In [ ]:
//index.ts

/** Initiate a new recording stream.
 * Calls startStreamingSession & processTextListener
 */
app.get('/startSession', async (_, res) => {
    const response = await startStreamingSession(revClient, revAiStream, bufferStream)
    if (response.connected) {
      outputPaths = setupFolders()
      
      processTextListener(
        transcriptionDataObject,
        revAiStream as Duplex,
        outputPaths
      )
      res.sendStatus(200)
    } else {
      res.status(400).send(`Error starting session: ${response.error}`)
    }
  })

In [ ]:
// call to gpt w/ context - node stream

//.. part of wake function
    // Get the formatted notes. Add in the transciptionDataObject. Send this to the gpt api
    let formattedFile = ''
    const ffPath = `${outputPaths.generated}/formatted_note.md`
    if (fs.existsSync(ffPath)) {
      formattedFile = fs.readFileSync(ffPath, 'utf-8')
    }
    const fChunk =
      formattedFile.length > 4000 ? formattedFile.slice(-4000) : formattedFile
    const extraData =
      transcriptionDataObject.finalData + transcriptionDataObject.partialData
    formatDocument(extraData, outputPaths)
    const chatResponse = await getAnswerStreaming(fChunk + extraData)

    // Convert gpt response stream to a text stream
    const textStream = new Transform({
      writableObjectMode: true,
      transform (chunk: ChatCompletionChunk, encoding, callback) {
        const finishReason = chunk.choices[0].finish_reason
        if (finishReason) {
          if (finishReason !== 'stop') console.log(finishReason)
          // End stream
          this.push(null)
          return callback()
        }
        this.push(chunk.choices[0].delta.content ?? 'ERR')
        callback()
      }
    })
    chatResponse?.pipe(textStream)
    // Text to speech (shown in next code chunk)
    const audioStream = textStreamToAudioStream(textStream)

In [ ]:

/** Create a passthrough, to pipe the audioStream through it. 
 * As text is recieved (in chunks of 100 or line end), run it through the TTS.
 * Write returned audio to the passthrough. When that audio ends, resume textStream
 * When the text stream ends, handle the last bit
 * Returns the readable passThroughStream
*/
export function textStreamToAudioStream (textStream: Readable): Readable {
    const passThroughStream = new PassThrough()
    let accumulatedText = ''
  
    textStream.on('data', async chunk => {
      accumulatedText += chunk.toString()
      if (accumulatedText.length > 100 || accumulatedText.includes('\n')) {
        textStream.pause() // Pause the text stream to wait for the current chunk to be processed
        try {
          const audioStream = (await convertTextToSpeech(
            accumulatedText
          )) as Readable
          audioStream.pipe(passThroughStream, { end: false })
          audioStream.on('end', () => {
            textStream.resume() // Resume the text stream after the chunk is processed
          })
        } catch (err) {
          passThroughStream.emit('error', err)
        }
        accumulatedText = ''
      }
    })
  
    textStream.on('end', async () => {
      if (accumulatedText) {
        try {
          const audioStream = (await convertTextToSpeech(
            accumulatedText
          )) as Readable
          audioStream.pipe(passThroughStream, { end: false })
          audioStream.on('end', () => {
            passThroughStream.end() // End when the last chunk is processed
          })
        } catch (err) {
          passThroughStream.emit('error', err)
        }
      } else {
        passThroughStream.end() // End if there's no remaining text
      }
    })
  
    textStream.on('error', err => {
      passThroughStream.emit('error', err)
    })
  
    return passThroughStream
  }

#### Cisco Hackathon

I competed in an internal Cisco hackathon to explore how we could improve  customer experience use cases with AI. 

My team won 1st place out of over 20 teams!

The compeotition took place over the span of two weeks. It was hosted by a different organization in Cisco, so it was different products and a different setting than I'm used to. Luckily my team had some experience we were able to draw on.

I had a lot of fun meeting people, collaborating, and creating our project. 


I played a few key roles on the team: 
1. Collaborating closely with our team lead to hash out his ideas and customer knowledge, and come up with a winning idea. He has a lot of knowledge around the specific field; *together* we were able to turn this into an implementation. 
2. Executing on this; I quickly learned how to build, and built, a smart AI assistant using Streamlit and a RAG system with FAISS and LangChain.

This showcases my holistic skill set. <span id="text-accent">I’m at my best when I have the ability to work closely with the product, with stakeholders, and with the implementation itself!</span>

My team lead graciously said this about me:

> "Daniel continuously demonstrated his ability to navigate high-level business strategy and translate that into ideas, concepts, code, and connections that support the achievement of those goals."

** add some more about this showing my ideation, prototyping, and ability to see the big picture, while also coding it! **
